In [1]:
import torch
import sys
sys.path.append('../')

from utils.synthetic import camera_lib
import numpy as np
import torch
import os
import shutil

In [2]:
ARKit_scense_path = '/mnt/IRONWOLF1/ashkan/data/ARKitScenes/upsampling'
split = ['Training', 'Validation']

In [3]:
val_video_ids = os.listdir(os.path.join(ARKit_scense_path, split[1]))
trian_video_ids = os.listdir(os.path.join(ARKit_scense_path, split[0]))

## Remove previous focal stack
Run this cell only if you want to remove the previous focal stack

In [4]:
def remove_focal_stack(video_ids, split):
    for video_id in video_ids:
        video_path = os.path.join(ARKit_scense_path, split, video_id, 'focal_stack')
        if os.path.exists(video_path):
            shutil.rmtree(video_path)
            print('Removed:', video_path)
        else:
            print('No focal stack in:', video_path)


In [5]:
# remove focal stack from validation set
remove_focal_stack(val_video_ids, split[1])
# remove focal stack from training set
remove_focal_stack(trian_video_ids, split[0])

No focal stack in: /mnt/IRONWOLF1/ashkan/data/ARKitScenes/upsampling/Validation/47334105/focal_stack
No focal stack in: /mnt/IRONWOLF1/ashkan/data/ARKitScenes/upsampling/Validation/47332005/focal_stack
No focal stack in: /mnt/IRONWOLF1/ashkan/data/ARKitScenes/upsampling/Validation/45261588/focal_stack
No focal stack in: /mnt/IRONWOLF1/ashkan/data/ARKitScenes/upsampling/Validation/47331262/focal_stack
No focal stack in: /mnt/IRONWOLF1/ashkan/data/ARKitScenes/upsampling/Validation/41069048/focal_stack
No focal stack in: /mnt/IRONWOLF1/ashkan/data/ARKitScenes/upsampling/Validation/42898826/focal_stack
No focal stack in: /mnt/IRONWOLF1/ashkan/data/ARKitScenes/upsampling/Validation/47333937/focal_stack
No focal stack in: /mnt/IRONWOLF1/ashkan/data/ARKitScenes/upsampling/Validation/47333934/focal_stack
No focal stack in: /mnt/IRONWOLF1/ashkan/data/ARKitScenes/upsampling/Validation/45662981/focal_stack
No focal stack in: /mnt/IRONWOLF1/ashkan/data/ARKitScenes/upsampling/Validation/41069021/fo

### Create Focal Stack directory


In [6]:
def create_focal_stack_dir(video_id, split):
    focal_stack_dir = os.path.join(
        ARKit_scense_path, split, video_id, 'focal_stack')
    if not os.path.exists(focal_stack_dir):
        os.makedirs(focal_stack_dir)
    return True

In [7]:
for video_id in val_video_ids:
    create_focal_stack_dir(video_id, split[1])

for video_id in trian_video_ids:
    create_focal_stack_dir(video_id, split[0])

# Generate focal stack


### Setup camera


In [8]:
renderer = camera_lib.GaussPSF(9)
renderer.cuda()
# camera = camera_lib.ThinLenCamera(1.8, focal_length=29e-3, pixel_size= 1.20e-4)
# camera = camera_lib.ThinLenCamera()


In [9]:
import imageio
import matplotlib.pyplot as plt

In [10]:
def gen_focal_stack(split, stack_size=10):
    video_id_list = trian_video_ids if split == 'Training' else val_video_ids

    for video_id in video_id_list:
        rgb_list = os.listdir(os.path.join(
            ARKit_scense_path, split, video_id, 'wide'))

        # check if the focal stack is already created
        focal_stack_dir = os.path.join(
            ARKit_scense_path, split, video_id, 'focal_stack')
        if len(os.listdir(focal_stack_dir)) > 0:
            print(f'Focal stack for {video_id} is already created')
            continue
        else:
            print(f'Creating focal stack for {video_id}')
            for rgb in rgb_list:
                # open the color image
                color_img = imageio.imread(os.path.join(
                    ARKit_scense_path, split, video_id, 'wide', rgb), pilmode="RGB")
                depth_img = imageio.imread(os.path.join(
                    ARKit_scense_path, split, video_id, 'highres_depth', rgb), pilmode="I")

                # Convert to float tensor
                color_img = torch.from_numpy(color_img).float()
                depth_img = torch.from_numpy(depth_img).float()
                depth_img = depth_img / 1000.0
                depth_img = depth_img.unsqueeze(0)

                color_img = color_img.permute(2, 0, 1)

                # Create the focal stack
                color_img = color_img/255.0
                min_depth = torch.min(depth_img)
                max_depth = torch.max(depth_img)

                focus_distances = torch.linspace(
                    min_depth, max_depth, steps=stack_size)

                focal_stack = camera_lib.render_defocus(
                    color_img,
                    depth_img,
                    camera,
                    renderer,
                    focus_distances
                )
                
                # save the focal stack
                focal_stack_dir = os.path.join(
                    ARKit_scense_path, split, video_id, 'focal_stack')
                for i in range(stack_size):
                    if os.path.exists(os.path.join(focal_stack_dir, f'{rgb[:-4]}')):
                        folder_dir = os.path.join(focal_stack_dir, f'{rgb[:-4]}')
                        imageio.imwrite(os.path.join(folder_dir, f'{i}.png'), (
                            focal_stack[i].permute(1, 2, 0).cpu().numpy()*255).astype(np.uint8))
                    else:
                        os.mkdir(os.path.join(focal_stack_dir, f'{rgb[:-4]}'))

In [11]:
gen_focal_stack(split[0], 10) # Validation

Creating focal stack for 47332015
Creating focal stack for 47429677
Creating focal stack for 42899202
Creating focal stack for 42446068
Creating focal stack for 43828345
Creating focal stack for 48458587
Creating focal stack for 47331175
Creating focal stack for 42899542
Creating focal stack for 45261532
Creating focal stack for 42445479
Creating focal stack for 47333838
Creating focal stack for 42445078
Creating focal stack for 47431155
Creating focal stack for 48018149
Creating focal stack for 43828391
Creating focal stack for 48458231
Creating focal stack for 47333168
Creating focal stack for 45261499
Creating focal stack for 47429888
Creating focal stack for 47332074
Creating focal stack for 47333574
Creating focal stack for 45261080
Creating focal stack for 44358257
Creating focal stack for 42898007
Creating focal stack for 42447210
Creating focal stack for 42444738
Creating focal stack for 47331144
Creating focal stack for 42447294
Creating focal stack for 48458249
Creating focal

KeyboardInterrupt: 

In [ ]:
gen_focal_stack(split[1], 10) # Training